<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/new_training_CNN_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traininig CNN Spectrum

# Loading Necesary Libraries

In [54]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
import re
import array
import seaborn as sns
from tqdm import tqdm

from functools import reduce
from random import random, gauss
from math import modf, pi, cos, sin, sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from sklearn.mixture import GaussianMixture
import os, time, math, csv, joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import os, time, math, csv, joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()

# Load Enviroment

In [55]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [56]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Change the number to change the paths.
worker = workers[0]

if worker == "Ronald":
  path = "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

# Create training/testing df

In [57]:
def rand(start, end, num): # get random value function
    res = []
    for j in range(num):
        res.append(np.random.randint(start, end))
    return res

In [58]:
def calc_freq_behavior(working_df, selected_point, multi=False): # Individual freq calculator
  extra_data = working_df.iloc[:, 0:6]
  freq_selected = working_df.iloc[:, selected_point+6]

  freq_data_df = pd.merge(extra_data, freq_selected, left_index=True, right_index=True)
  freq_data_df['Distance_km'] = freq_data_df['Distance_km'].astype(int)

  headers = list(freq_data_df.columns.values)
  selected_val = round(float(headers[-1]), 5)

  distances = [x*80 for x in range(1, 26)]
  freq_behavior = {}
  f_behavior = {}

  for dist in distances: # loop all distances available

    distance_df = freq_data_df[(freq_data_df['Distance_km'] == dist)] # select all the row with current eval distance
    freq_distance_vals = distance_df[str(selected_val)].tolist()

    freq_mean = np.mean(freq_distance_vals)
    freq_std = np.std(freq_distance_vals)
    results = (freq_mean, freq_std)
    f_behavior[dist] = results

  freq_behavior[selected_val] = f_behavior
  return freq_behavior

In [59]:
def select_eval_cols(working_df, eval_cols_param): # retturn index of column to be evaluated
  if eval_cols_param[0] == 'even': 
    break_point = int((working_df.shape[1]-6)/(eval_cols_param[1]+1))
    selected_cols = []
    for col_idx in range(1, eval_cols_param[1]+1):
      selected_cols.append((col_idx*break_point)-1)
  else:
    selected_cols = rand(1, working_df.shape[1]-6, eval_cols_param[1])
    selected_cols = sorted(selected_cols)
  return selected_cols

In [68]:
# multi_params = (method, number_freq_to_analyze)
def multiple_freq_analyzer(working_df, multi_params): #method = 'rand' or 'even' (multiple freq behavior calculator)
  selected_cols = select_eval_cols(working_df, multi_params)

  multi_freq_behavior = {}
  for selected_point in tqdm(selected_cols):
    freq_behavior = calc_freq_behavior(working_df, selected_point)
    selected_v = [v for v in freq_behavior.keys()]

    selected_d = {}
    for data in freq_behavior.values():
      for k, v in data.items():
        selected_d[k] = v

    multi_freq_behavior[selected_v[0]] = selected_d
  return multi_freq_behavior

In [61]:
def freq_dict_to_df(multi_freq_behavior):
  distances = [x*80 for x in range(1, 26)]
  freq_mean_data = {'distances':distances}
  freq_std_data = {'distances':distances}

  for header, distances_dict in multi_freq_behavior.items():
    col_data_mean = []
    col_data_std = []
    for distance, results in distances_dict.items():
      col_data_mean.append(results[0])
      col_data_std.append(results[1])
    freq_mean_data[header] = col_data_mean
    freq_std_data[header] = col_data_std

  freq_mean_df = pd.DataFrame(freq_mean_data)
  freq_std_df = pd.DataFrame(freq_std_data)
  
  return freq_mean_df, freq_std_df

## Pearson Correletion

In [70]:
#eval_method = 'avg' or 'min' or 'max'

def pearson_matrix(working_df , source_sample_id, eval_method = 'avg'):
  distances = [x*80 for x in range(1, 26)]
  result_data = {'source/target': distances}
  
  print('Calcylating Pearson values using ({}) method:'.format(eval_method))
  for target_dist in tqdm(distances): # loop all distances available
    result_target = []
    for source_dist in distances: # loop all distances available
      source_df = working_df[(working_df['Distance_km'] == source_dist) & (working_df['PBRS_id'] == source_sample_id)]# select data of source (distance and sample_id) 
      source_data = source_df.iloc[0, 6:source_df.shape[1]].tolist()
      pearson_values = []
      for target_sample_id in working_df['PBRS_id'].unique(): # loop all samples id to then exclude the source sample id
        if target_sample_id != source_sample_id:
          target_df = working_df[(working_df['Distance_km'] == target_dist) & (working_df['PBRS_id'] == target_sample_id)] # select data of target
          target_data = target_df.iloc[0, 6:target_df.shape[1]].tolist()
          pearson_values.append(round(pearsonr(source_data, target_data)[0], 5)) # pearson correlation
      if eval_method == 'avg':
        p_result = np.mean(pearson_values)
      elif eval_method == 'max':
        p_result = np.max(pearson_values)
      elif eval_method == 'min':
        p_result = np.min(pearson_values)
      result_target.append(p_result)
    result_data[str(eval_method)+'_'+str(target_dist)] = result_target

  pearson_matrix_df = pd.DataFrame.from_dict(result_data)
  #pearson_matrix_df = pearson_matrix_df.set_index('source/target')

  return pearson_matrix_df, result_data

## Mean/std analysis

In [63]:
def calc_freq_behavior(working_df, selected_point): # Individual freq calculator
  extra_data = working_df.iloc[:, 0:6]
  freq_selected = working_df.iloc[:, selected_point+6]

  freq_data_df = pd.merge(extra_data, freq_selected, left_index=True, right_index=True)
  freq_data_df['Distance_km'] = freq_data_df['Distance_km'].astype(int)

  headers = list(freq_data_df.columns.values)
  selected_val = round(float(headers[-1]), 5)

  distances = [x*80 for x in range(1, 26)]
  freq_behavior = {}
  f_behavior = {}

  for dist in distances: # loop all distances available

    distance_df = freq_data_df[(freq_data_df['Distance_km'] == dist)] # select all the row with current eval distance
    freq_distance_vals = distance_df[str(selected_val)].tolist()

    freq_mean = np.mean(freq_distance_vals)
    freq_std = np.std(freq_distance_vals)
    results = (freq_mean, freq_std)
    f_behavior[dist] = results
  freq_behavior[selected_val] = f_behavior
  return freq_behavior

In [64]:
def calc_freq_behavior_v2(working_df, amt_of_selected_point): # Individual freq calculator
  extra_data = working_df.iloc[:, 0:6]

  distances = [x*80 for x in range(1, 26)]

  std_combined = []
  for dist in distances: # loop all distances available
    std_all = []
    for val_idx in range(0, amt_of_selected_point):

      freq_selected = working_df.iloc[:, val_idx+6]
      freq_data_df = pd.merge(extra_data, freq_selected, left_index=True, right_index=True)
      freq_data_df['Distance_km'] = freq_data_df['Distance_km'].astype(int)

      headers = list(freq_data_df.columns.values)
      selected_val = round(float(headers[-1]), 5)
      
      distance_df = freq_data_df[(freq_data_df['Distance_km'] == dist)] # select all the row with current eval distance
      freq_distance_vals = distance_df[str(selected_val)].tolist()

      freq_std = np.std(freq_distance_vals)
      std_all.append(freq_std)
      std_avg = round(np.average(std_all), 5)
    std_combined.append(std_avg)


  data = {'source/target':distances, 'std':std_combined}
 
  # Create DataFrame
  std_df = pd.DataFrame(data)
  return std_df

In [73]:
# Get features 
def get_features(eval_freq_data_df, amt_sel_points):
  methods = ['min', 'avg', 'max']
  dfs = []
  for m in methods:
    pearson_matrix_df, result_data = pearson_matrix(eval_freq_data_df , source_sample_id=1, eval_method = m)
    dfs.append(pearson_matrix_df)

  final_df = reduce(lambda  left,right: pd.merge(left,right,on=['source/target'],how='outer'), dfs)

  test_freq_data_df = calc_freq_behavior(eval_freq_data_df, selected_point=1)

  std_df = calc_freq_behavior_v2(eval_freq_data_df, amt_of_selected_point = amt_sel_points)

  final_final_df = pd.merge(final_df, std_df, on = "source/target", how = "inner")
  return final_final_df

In [72]:
def train_test_split(working_df, amt_train = 0.8):
  print('Creating train/test plit')
  num_sample_train = math.ceil((working_df.shape[1]-6)*amt_train)
  num_sample_test = (working_df.shape[1]-6) - num_sample_train

  multi_params = ('rand', num_sample_train)  # (evenly separate points(other option 'rand'for randomly selected points), mount of points to be selected)
  multi_freq_behavior = multiple_freq_analyzer(working_df, multi_params)

  eval_points = select_eval_cols(working_df, multi_params) # get indexes of of points to be evaluated
  real_eval_idx = [eval_point+6 for eval_point in eval_points]

  extra_data = working_df.iloc[:, 0:6]
  eval_freq_df = working_df.iloc[:, real_eval_idx]
  train_df = pd.merge(extra_data, eval_freq_df, left_index=True, right_index=True) #df we only eval freqs TRAIN

  test_df= working_df.copy()
  test_df.drop(test_df.columns[real_eval_idx], axis=1, inplace=True) # df without eval freqs TEST
  return train_df, test_df

In [75]:
input_path = path + "/Spectrum/CNN/working_df.csv"

working_df = pd.read_csv(input_path)
working_df['Distance_km'] = working_df['Distance_km'].astype(int)

train_df, test_df = train_test_split(working_df, amt_train = 0.8)
print('------------------------------')
# Get features 
print('Calculating features for train')
train_features_df = get_features(train_df, train_df.shape[1]-6)
print('------------------------------')
print('Calculating deatures for test')
test_features_df = get_features(test_df, test_df.shape[1]-6)

#final_final_df.to_csv(path+'/Spectrum/CNN/training_data_DNN.csv')

Creating train/test plit


100%|██████████| 1654/1654 [00:28<00:00, 58.46it/s]


------------------------------
Calculating features for train
Calcylating Pearson values using (min) method:


100%|██████████| 25/25 [00:22<00:00,  1.10it/s]


Calcylating Pearson values using (avg) method:


...

100%|██████████| 25/25 [00:13<00:00,  1.91it/s]


Calcylating Pearson values using (avg) method:


100%|██████████| 25/25 [00:13<00:00,  1.91it/s]


Calcylating Pearson values using (max) method:


100%|██████████| 25/25 [00:13<00:00,  1.92it/s]


In [76]:
train_features_df

,source/target,min_80,min_160,min_240,min_320,min_400,min_480,min_560,min_640,min_720,...,max_1360,max_1440,max_1520,max_1600,max_1680,max_1760,max_1840,max_1920,max_2000,std
0,80,-0.04108,-0.01288,0.00367,-0.00945,-0.04368,-0.01738,0.01660,0.00485,-0.02249,...,0.11473,0.09477,0.10666,0.13407,0.90497,0.12377,0.08940,0.11137,0.11519,3.55079
1,160,-0.04953,-0.04477,-0.06493,-0.04014,-0.00259,-0.03320,-0.06212,-0.02932,-0.02126,...,0.95935,0.06965,0.09083,0.13767,0.08283,0.89931,0.08128,0.10298,0.12906,3.47441
2,240,-0.01424,-0.00992,0.01218,0.01479,-0.00786,-0.01270,0.01837,-0.01070,0.00869,...,0.09997,0.95865,0.14753,0.14295,0.12217,0.09422,0.90036,0.14837,0.15511,3.37690
3,320,-0.00446,-0.02357,0.02544,-0.01309,-0.01943,-0.00314,0.00322,0.04090,0.01800,...,0.11914,0.14159,0.95383,0.10020,0.09629,0.12263,0.14098,0.89888,0.10746,3.49152
4,400,0.01330,-0.03422,-0.02684,-0.06426,0.00859,0.03098,-0.03601,-0.04401,-0.03979,...,0.06851,0.11905,0.10895,0.95870,0.10856,0.06911,0.12036,0.11809,0.90424,3.46657
5,480,-0.05660,0.05081,-0.04209,-0.00900,-0.03715,-0.02743,0.03417,-0.00718,-0.02904,...,0.18584,0.13975,0.11427,0.11654,0.95990,0.18793,0.12348,0.12289,0.13859,3.54080
6,560,0.01123,0.01279,-0.04274,-0.06185,-0.03677,0.00413,0.01090,-0.05297,-0.07242,...,0.92020,0.06876,0.05707,0.10875,0.10160,0.95614,0.07654,0.07104,0.10498,3.47194
7,640,-0.04510,0.00376,-0.04704,0.01571,-0.01040,-0.05447,-0.00325,-0.03499,0.01399,...,0.08285,0.90464,0.08844,0.12722,0.07152,0.07773,0.95253,0.07733,0.12827,3.36992
8,720,-0.00812,-0.05782,0.03200,-0.00201,-0.00609,0.01240,-0.03984,0.02560,0.01188,...,0.05956,0.07659,0.92380,0.10412,0.09880,0.05832,0.07173,0.95945,0.10457,3.46960
9,800,-0.01486,0.03863,0.00557,-0.05748,-0.00432,0.01930,0.02502,-0.01352,-0.08049,...,0.14867,0.13357,0.13715,0.91755,0.13399,0.13067,0.12642,0.12208,0.96051,3.44016


In [77]:
test_features_df

,source/target,min_80,min_160,min_240,min_320,min_400,min_480,min_560,min_640,min_720,...,max_1360,max_1440,max_1520,max_1600,max_1680,max_1760,max_1840,max_1920,max_2000,std
0,80,0.00723,-0.04392,0.03682,-0.01765,-0.03278,-0.00272,-0.02505,0.02643,-0.00132,...,0.14696,0.13407,0.13530,0.11217,0.89539,0.13370,0.12893,0.13280,0.10844,3.43875
1,160,-0.03632,-0.00322,-0.04452,-0.01108,0.01402,-0.05438,-0.00997,-0.01445,-0.00005,...,0.95243,0.06865,0.07639,0.11565,0.07893,0.90708,0.06543,0.08606,0.11042,3.41853
2,240,0.02617,-0.01825,0.03009,0.02250,-0.03250,0.04410,0.01326,0.02434,0.03200,...,0.15235,0.96649,0.17185,0.14057,0.14464,0.13898,0.92001,0.18902,0.14121,3.38613
3,320,-0.01508,-0.05124,-0.01394,-0.06319,-0.05637,-0.00052,-0.03519,0.01195,-0.04934,...,0.05694,0.07197,0.95893,0.05842,0.10379,0.05456,0.07407,0.90281,0.05511,3.51938
4,400,-0.00254,0.00263,0.00222,-0.01868,-0.04659,0.01836,0.03145,0.00956,-0.01798,...,0.13664,0.13317,0.07736,0.95500,0.11445,0.12548,0.11657,0.08541,0.91074,3.53102
5,480,-0.00332,-0.01519,0.01355,-0.01747,0.01425,-0.02135,0.00050,0.01726,-0.03852,...,0.11758,0.10595,0.10361,0.10511,0.95842,0.13132,0.12412,0.11021,0.09654,3.46280
6,560,-0.02037,-0.05049,-0.02084,-0.07732,-0.07129,0.00213,-0.05937,-0.03445,-0.08594,...,0.92379,0.10305,0.06940,0.07350,0.07283,0.96011,0.09643,0.05820,0.06968,3.42881
7,640,-0.06679,-0.00878,-0.01500,0.01397,-0.04368,-0.05578,-0.01861,-0.00199,0.00357,...,0.06455,0.92186,0.11120,0.09844,0.09460,0.05590,0.95921,0.11006,0.09630,3.41174
8,720,-0.07878,0.01412,-0.02532,-0.04032,-0.01303,-0.05470,0.00479,-0.03383,-0.05738,...,0.07806,0.12967,0.92315,0.09706,0.10288,0.08125,0.12720,0.96035,0.09636,3.51243
9,800,-0.04689,-0.02521,-0.01045,0.01432,-0.03614,-0.03322,-0.03600,-0.03284,0.00677,...,0.12172,0.11063,0.13173,0.92487,0.08599,0.10882,0.12446,0.13098,0.96465,3.52205


# Neural Network

## Pre-processing

In [ ]:
df=pd.read_csv(path+'/Spectrum/CNN/training_data_DNN.csv')

In [ ]:
Y = df.iloc[:, 1].to_numpy().reshape(-1,1)
X = df.iloc[:, 2:df.shape[1]]

sc_input = MinMaxScaler()
sc_output = MinMaxScaler()
X_train = sc_input.fit_transform(X) # convert features to values from 0 to 1
Y_train = sc_output.fit_transform(Y) # convert distances to values from 0 to 1

## Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


def ann():
    model = Sequential()
    model.add(Dense(24, input_dim = 24, activation = 'tanh'))
    model.add(Dense(12,activation='tanh'))
    model.add(Dense(6,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=100,batch_size=32)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

## Test Model

In [ ]:
start_time = time.time()
Y_test_pred=model_ann.predict(X_test)

time_eval_ann=time.time()-start_time

real=list(list(zip(*Y_test))[0])

#real=list(map(list, zip(*Y_train)))
pred=list(Y_test_pred)
print(pred)


dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

#np.add = add values in a array
#np.multiply multiple values in a array
print(real)
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
#print(real_abs)
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar
#print(pred_abs)

#np.divide = divide values in a array
error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))

res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res.to_csv("results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()